# V26: Dilution / host plausibility and planet-radius range

This notebook combines:
- Gaia neighbor information (crowding / flux fractions),
- the depth range observed under reasonable preprocessing choices (V24), and
- simple dilution physics

to estimate:
- a plausible **planet radius range** (assuming the target is the host), and
- whether alternative nearby-host scenarios look physically plausible.

This is a *metrics-only* plausibility check. It does not replace pixel-level localization.


In [ ]:
from __future__ import annotations

import json
import math
import sys
from pathlib import Path

import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh
import bittr_tess_vetter.api as btv
from bittr_tess_vetter.api.catalogs import query_gaia_by_id_sync, query_gaia_by_position_sync

from bittr_tess_vetter.api.stellar_dilution import (
    build_host_hypotheses_from_profile,
    compute_dilution_scenarios,
    evaluate_physics_flags,
)

step_id = '40_v26_dilution_and_radius_range'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

# Depth range from V24 artifact (keeps this notebook tied to the actual baseline used)
v24_path = Path(
    'docs/tutorials/artifacts/tutorial_toi-5807-incremental/38_v24_triceratops_fpp_sensitivity_grid/'
    'triceratops_fpp_sensitivity_grid.json'
)
v24 = json.loads(v24_path.read_text())
depths = [float(r['depth_ppm_used']) for r in v24['rows']]
depth_min = float(min(depths))
depth_max = float(max(depths))
depth_mid = float(np.median(depths))

# Query Gaia (network)
gaia = query_gaia_by_position_sync(
    ra=float(sh.RA_DEG),
    dec=float(sh.DEC_DEG),
    radius_arcsec=60.0,
)

primary_g = gaia.source.phot_g_mean_mag if gaia.source is not None else None
primary_radius = float(sh.STELLAR.radius)

# Select a manageable set of potentially important companions.
# (Close + bright in Gaia G; then try to fetch radii for those IDs.)
neighbors = list(gaia.neighbors)
neighbors.sort(key=lambda n: (n.separation_arcsec, n.delta_mag if n.delta_mag is not None else 99.0))
candidates = [n for n in neighbors if (n.delta_mag is not None and n.delta_mag <= 6.0 and n.separation_arcsec <= 60.0)]
candidates = candidates[:10]

close_bright_companions = []
for n in candidates:
    radius_rsun = None
    try:
        prof = query_gaia_by_id_sync(int(n.source_id), cone_radius_arcsec=0.0)
        if prof.astrophysical is not None and prof.astrophysical.radius_gspphot is not None:
            radius_rsun = float(prof.astrophysical.radius_gspphot)
    except Exception:
        radius_rsun = None

    close_bright_companions.append(
        (
            int(n.source_id),
            float(n.separation_arcsec),
            float(n.phot_g_mean_mag) if n.phot_g_mean_mag is not None else None,
            float(n.delta_mag) if n.delta_mag is not None else None,
            radius_rsun,
        )
    )

primary_h, companions_h = build_host_hypotheses_from_profile(
    tic_id=int(sh.TIC_ID),
    primary_g_mag=float(primary_g) if primary_g is not None else None,
    primary_radius_rsun=float(primary_radius),
    close_bright_companions=close_bright_companions,
)

def planet_radius_rearth(depth_ppm: float, rstar_rsun: float) -> float:
    # depth ≈ (Rp/Rs)^2
    rp_rs = math.sqrt(max(0.0, float(depth_ppm) / 1e6))
    rsun_to_rearth = 109.076
    return float(rp_rs * float(rstar_rsun) * rsun_to_rearth)

rp_min = planet_radius_rearth(depth_min, primary_radius)
rp_mid = planet_radius_rearth(depth_mid, primary_radius)
rp_max = planet_radius_rearth(depth_max, primary_radius)

sc_min = compute_dilution_scenarios(observed_depth_ppm=depth_min, primary=primary_h, companions=companions_h)
sc_mid = compute_dilution_scenarios(observed_depth_ppm=depth_mid, primary=primary_h, companions=companions_h)
sc_max = compute_dilution_scenarios(observed_depth_ppm=depth_max, primary=primary_h, companions=companions_h)

scenarios = {
    'depth_min': [s.to_dict() for s in sc_min],
    'depth_mid': [s.to_dict() for s in sc_mid],
    'depth_max': [s.to_dict() for s in sc_max],
}

# Summarize flags using the median-depth scenarios.
flags = evaluate_physics_flags(sc_mid, host_ambiguous=(len(companions_h) > 0))

out = {
    'depth_ppm_range_from_v24': {'min': depth_min, 'median': depth_mid, 'max': depth_max},
    'planet_radius_rearth_if_on_target': {'min': rp_min, 'median': rp_mid, 'max': rp_max, 'stellar_radius_rsun': primary_radius},
    'gaia_primary_g_mag': float(primary_g) if primary_g is not None else None,
    'n_gaia_neighbors_within_60_arcsec': int(len(neighbors)),
    'companions_used': [
        {
            'source_id': sid,
            'sep_arcsec': sep,
            'g_mag': gmag,
            'delta_mag': dmag,
            'radius_rsun': r,
        }
        for sid, sep, gmag, dmag, r in close_bright_companions
    ],
    'primary_hypothesis': primary_h.to_dict(),
    'companion_hypotheses': [h.to_dict() for h in companions_h],
    'scenarios': scenarios,
    'physics_flags': flags.to_dict() if hasattr(flags, 'to_dict') else dict(flags),
}

json_name = 'dilution_and_radius_range.json'
(run_out_dir / json_name).write_text(json.dumps(out, indent=2, sort_keys=True))
if docs_out_dir is not None:
    (docs_out_dir / json_name).write_text(json.dumps(out, indent=2, sort_keys=True))

print(json.dumps({
  'depth_ppm_range_from_v24': out['depth_ppm_range_from_v24'],
  'planet_radius_rearth_if_on_target': out['planet_radius_rearth_if_on_target'],
  'physics_flags': out['physics_flags'],
  'companions_used_n': len(close_bright_companions),
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

This notebook writes a full JSON payload to:
- `docs/tutorials/artifacts/tutorial_toi-5807-incremental/40_v26_dilution_and_radius_range/dilution_and_radius_range.json`

Depth and Gaia neighbor counts can change if upstream catalogs change.

```text
{
  "depth_ppm_range_from_v24": {
    "max": 240.51652026335634,
    "median": 221.56494376329715,
    "min": 199.7750411103505
  },
  "planet_radius_rearth_if_on_target": {
    "max": 2.791165596624512,
    "median": 2.6789442198133018,
    "min": 2.5438046790402646,
    "stellar_radius_rsun": 1.65
  },
  "physics_flags": {
    "planet_radius_inconsistent": false,
    "rationale": "Transit depth consistent with planetary interpretation",
    "requires_resolved_followup": false
  }
}
```

</details>


In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

# Plot the AO contrast curve alongside Gaia neighbors (delta_mag vs separation).
cc = btv.load_contrast_curve_exofop_tbl('docs/tutorials/data/tic188646744/PHARO_Kcont_plot.tbl', filter='Kcont')

fig, ax = plt.subplots(figsize=(7.5, 4.5), dpi=150)
ax.plot(cc.separation_arcsec, cc.delta_mag, color='black', lw=2, label='AO contrast curve (Kcont)')

# Gaia neighbors in G band (approximate overlay; different bandpass)
x = [n.separation_arcsec for n in neighbors if n.delta_mag is not None]
y = [n.delta_mag for n in neighbors if n.delta_mag is not None]
ax.scatter(x, y, s=18, alpha=0.6, label='Gaia neighbors (ΔG)')

ax.set_xlabel('Separation (arcsec)')
ax.set_ylabel('Delta mag')
ax.set_title('AO contrast curve vs Gaia neighbor brightness (bandpasses differ)')
ax.grid(True, alpha=0.2)
ax.legend(loc='lower right')
fig.tight_layout()

step_id = '40_v26_dilution_and_radius_range'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
png_name = 'V26_ao_contrast_vs_gaia_neighbors.png'
run_png = run_out_dir / png_name
fig.savefig(run_png)
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / png_name)
plt.close(fig)

print(json.dumps({'run_plot_path': str(run_png), 'docs_plot_path': str((docs_out_dir / png_name) if docs_out_dir is not None else None)}, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/40_v26_dilution_and_radius_range/V26_ao_contrast_vs_gaia_neighbors.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Planet radius range (target host):** computed from the V24 depth range and the assumed stellar radius using depth≈(Rp/Rs)^2.
- **Dilution scenarios:** the `scenarios` table reports the flux fraction each hypothesized host would contribute (from Gaia magnitudes) and the corresponding corrected depth.
- **How to use this:**
  - If reasonable dilution corrections still imply a planet-sized companion on the target, that supports plausibility.
  - If nearby-host scenarios require extreme depths or implausibly large companions (when radii are available), those hypotheses are disfavored.
- **Caveats:**
  - Gaia `ΔG` and AO `ΔKcont` are not the same bandpass; the overlay plot is qualitative.
  - Host plausibility is not a substitute for pixel localization; it is a physics/photometry sanity check.

</details>
